In [1]:
%load_ext autoreload
import sys
import torch

import tqdm
from nntools.utils import Config
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

sys.path.append('../')
from experiment import OCTClassification


sys.path.append('../LRP_decomposition/LRP')


In [2]:
config_path = '../config.yaml'
config = Config(config_path)
config['Network']['architecture'] = 'ResNet152'
experiment = OCTClassification(config)



In [3]:


path = '/home/clement/Documents/Clement/runs/mlruns/1/f225fafe580e48d491e1eebe122d0635/artifacts/last.pth'
model = experiment.model
model.load(path)
model = model.network
model = model.eval()

Loading model from  /home/clement/Documents/Clement/runs/mlruns/1/f225fafe580e48d491e1eebe122d0635/artifacts/last.pth


In [5]:
%autoreload 2
sys.path.append('../Deep-Taylor-Decomposition')

from model import saliency_mapping as sa_map

dataset = experiment.test_dataset

def show_cam_on_image(img, mask, alpha):
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap*alpha + (1-alpha)*np.float32(img)
    cam = cam / np.max(cam)
    return cam

model_archi = 'resnet'
module_list = sa_map.model_flattening(model)
act_store_model = sa_map.ActivationStoringNet(module_list)
DTD = sa_map.DTD(-2.1179, 2.64)


index = np.arange(1000)
np.random.shuffle(index)    
output_vit = 'Explained-ResNet-DTD/'
labels = {} 
for k, v in dataset.map_class.items():
    labels[v] = k
if not os.path.exists(output_vit):
    os.makedirs(output_vit)
existing_files = os.listdir(output_vit)


def explain_map():
    for i in tqdm.tqdm(index):
        img, gt = dataset[i]
        filename = dataset.filename(i)
        img = img.unsqueeze(0)
        module_stack, output = act_store_model(img)
        saliency_map = DTD(module_stack, output, 4, model_archi)
        # saliency_map = torch.exp(torch.sum(saliency_map, dim=1).squeeze())
        # saliency_map = (saliency_map - saliency_map.min())/(saliency_map.max() - saliency_map.min())
        image = img.cpu().detach().squeeze().numpy().transpose((1,2,0))

        saliency_map = saliency_map.sum(1)
        saliency_map = saliency_map[0].detach().numpy()
        saliency_map = (saliency_map - saliency_map.min())/(saliency_map.max() - saliency_map.min())

        image = (image-image.min())/(image.max()-image.min())
        mix = show_cam_on_image(image, saliency_map, 1.0)
        
        filepath = dataset.img_filepath[i] 
        inp = cv2.imread(filepath)
        mix = cv2.resize(mix, dsize=inp.shape[:2][::-1])\
        
        cv2.imwrite(os.path.join(output_vit, filename), (255*mix).astype(np.uint8))

explain_map()


 30%|███       | 303/1000 [06:34<15:03,  1.30s/it]


 60%|██████    | 605/1000 [13:09<08:37,  1.31s/it]


 91%|█████████ | 907/1000 [19:48<02:06,  1.36s/it]


100%|██████████| 1000/1000 [21:51<00:00,  1.32s/it]


In [37]:
t_dataset = experiment.train_dataset
t_dataloader = experiment.get_dataloader(t_dataset, shuffle=False)
t_dataset.return_indices = True

In [41]:
filenames = []
for j, batch in (enumerate(tqdm.tqdm(t_dataloader[0]))):
    img, gt, idx = batch
    filenames += (t_dataset.filename(idx))




  0%|          | 0/3370 [00:00<?, ?it/s]


  0%|          | 1/3370 [00:00<32:38,  1.72it/s]


  0%|          | 5/3370 [00:00<23:22,  2.40it/s]


  0%|          | 9/3370 [00:00<17:19,  3.23it/s]


  0%|          | 13/3370 [00:01<13:06,  4.27it/s]


  1%|          | 17/3370 [00:01<10:00,  5.59it/s]


  1%|          | 21/3370 [00:01<07:51,  7.10it/s]


  1%|          | 25/3370 [00:01<06:22,  8.74it/s]


  1%|          | 29/3370 [00:02<05:19, 10.47it/s]


  1%|          | 33/3370 [00:02<04:36, 12.08it/s]


  1%|          | 37/3370 [00:02<04:06, 13.55it/s]


  1%|          | 41/3370 [00:02<03:48, 14.56it/s]


  1%|▏         | 45/3370 [00:02<03:33, 15.60it/s]

197it [00:27, 18.61it/s]


  1%|▏         | 49/3370 [00:03<03:20, 16.53it/s]


  2%|▏         | 53/3370 [00:03<03:11, 17.30it/s]


  2%|▏         | 57/3370 [00:03<03:07, 17.64it/s]


  2%|▏         | 61/3370 [00:03<03:04, 17.91it/s]


  2%|▏         | 65/3370 [00:03<03:00, 18.28it/s]


  2%|▏         | 69/3370 [00:04<03:00, 18.24it/

 16%|█▌        | 525/3370 [00:28<02:35, 18.35it/s]


 16%|█▌        | 529/3370 [00:28<02:34, 18.39it/s]


 16%|█▌        | 533/3370 [00:29<02:33, 18.51it/s]


 16%|█▌        | 537/3370 [00:29<02:32, 18.54it/s]


 16%|█▌        | 541/3370 [00:29<02:33, 18.46it/s]


 16%|█▌        | 545/3370 [00:29<02:30, 18.81it/s]


 16%|█▋        | 549/3370 [00:29<02:27, 19.07it/s]


 16%|█▋        | 553/3370 [00:30<02:30, 18.69it/s]


 17%|█▋        | 557/3370 [00:30<02:31, 18.58it/s]


 17%|█▋        | 561/3370 [00:30<02:32, 18.40it/s]


 17%|█▋        | 565/3370 [00:30<02:31, 18.54it/s]


 17%|█▋        | 569/3370 [00:31<02:29, 18.70it/s]


 17%|█▋        | 573/3370 [00:31<02:30, 18.59it/s]


 17%|█▋        | 577/3370 [00:31<02:30, 18.51it/s]


 17%|█▋        | 581/3370 [00:31<02:30, 18.59it/s]


 17%|█▋        | 585/3370 [00:31<02:29, 18.66it/s]


 17%|█▋        | 589/3370 [00:32<02:28, 18.74it/s]


 18%|█▊        | 593/3370 [00:32<02:28, 18.68it/s]


 18%|█▊        | 597/3370 [00:32<02:29, 18.55i

 29%|██▉       | 969/3370 [01:13<14:27,  2.77it/s]


 29%|██▉       | 970/3370 [01:13<19:44,  2.03it/s]


 29%|██▉       | 971/3370 [01:14<17:26,  2.29it/s]


 29%|██▉       | 973/3370 [01:14<14:47,  2.70it/s]


 29%|██▉       | 974/3370 [01:15<15:50,  2.52it/s]


 29%|██▉       | 975/3370 [01:15<17:17,  2.31it/s]


 29%|██▉       | 977/3370 [01:16<14:17,  2.79it/s]


 29%|██▉       | 978/3370 [01:16<14:00,  2.85it/s]


 29%|██▉       | 979/3370 [01:16<16:38,  2.39it/s]


 29%|██▉       | 981/3370 [01:17<12:58,  3.07it/s]


 29%|██▉       | 982/3370 [01:17<15:11,  2.62it/s]


 29%|██▉       | 983/3370 [01:18<16:55,  2.35it/s]


 29%|██▉       | 985/3370 [01:18<12:39,  3.14it/s]


 29%|██▉       | 986/3370 [01:19<19:52,  2.00it/s]


 29%|██▉       | 987/3370 [01:19<20:05,  1.98it/s]


 29%|██▉       | 990/3370 [01:20<18:19,  2.16it/s]


 29%|██▉       | 991/3370 [01:21<17:13,  2.30it/s]


 29%|██▉       | 994/3370 [01:22<16:16,  2.43it/s]


 30%|██▉       | 998/3370 [01:23<15:17,  2.58i

 39%|███▊      | 1301/3370 [03:14<08:58,  3.84it/s]


 39%|███▊      | 1302/3370 [03:16<19:53,  1.73it/s]


 39%|███▉      | 1306/3370 [03:17<16:51,  2.04it/s]


 39%|███▉      | 1307/3370 [03:17<13:52,  2.48it/s]


 39%|███▉      | 1308/3370 [03:18<16:30,  2.08it/s]


 39%|███▉      | 1310/3370 [03:18<13:23,  2.57it/s]


 39%|███▉      | 1311/3370 [03:19<14:38,  2.34it/s]


 39%|███▉      | 1312/3370 [03:20<29:30,  1.16it/s]


 39%|███▉      | 1316/3370 [03:22<24:23,  1.40it/s]


 39%|███▉      | 1320/3370 [03:23<20:19,  1.68it/s]


 39%|███▉      | 1324/3370 [03:25<18:01,  1.89it/s]


 39%|███▉      | 1328/3370 [03:26<16:33,  2.06it/s]


 40%|███▉      | 1332/3370 [03:28<15:55,  2.13it/s]


 40%|███▉      | 1336/3370 [03:29<14:45,  2.30it/s]


 40%|███▉      | 1340/3370 [03:31<14:24,  2.35it/s]


 40%|███▉      | 1344/3370 [03:33<13:56,  2.42it/s]


 40%|████      | 1348/3370 [03:34<13:32,  2.49it/s]


 40%|████      | 1352/3370 [03:35<12:41,  2.65it/s]


 40%|████      | 1356/3370 [

 50%|████▉     | 1675/3370 [05:36<18:38,  1.52it/s]


 50%|████▉     | 1678/3370 [05:37<15:07,  1.86it/s]


 50%|████▉     | 1679/3370 [05:37<14:17,  1.97it/s]


 50%|████▉     | 1682/3370 [05:39<13:22,  2.10it/s]


 50%|████▉     | 1683/3370 [05:39<12:45,  2.20it/s]


 50%|█████     | 1686/3370 [05:40<11:38,  2.41it/s]


 50%|█████     | 1687/3370 [05:40<11:36,  2.42it/s]


 50%|█████     | 1690/3370 [05:41<10:09,  2.76it/s]


 50%|█████     | 1691/3370 [05:42<14:46,  1.89it/s]


 50%|█████     | 1694/3370 [05:42<11:03,  2.52it/s]


 50%|█████     | 1695/3370 [05:44<19:53,  1.40it/s]


 50%|█████     | 1698/3370 [05:44<14:48,  1.88it/s]


 50%|█████     | 1699/3370 [05:45<18:18,  1.52it/s]


 51%|█████     | 1702/3370 [05:45<13:28,  2.06it/s]


 51%|█████     | 1703/3370 [05:46<19:12,  1.45it/s]


 51%|█████     | 1706/3370 [05:47<14:08,  1.96it/s]


 51%|█████     | 1707/3370 [05:48<19:41,  1.41it/s]


 51%|█████     | 1710/3370 [05:48<14:08,  1.96it/s]


 51%|█████     | 1712/3370 [

 59%|█████▉    | 1993/3370 [07:35<10:36,  2.16it/s]


 59%|█████▉    | 1997/3370 [07:36<09:45,  2.34it/s]


 59%|█████▉    | 2001/3370 [07:37<09:02,  2.52it/s]


 59%|█████▉    | 2005/3370 [07:39<08:40,  2.62it/s]


 60%|█████▉    | 2009/3370 [07:40<08:49,  2.57it/s]


 60%|█████▉    | 2013/3370 [07:42<08:28,  2.67it/s]


 60%|█████▉    | 2017/3370 [07:43<08:11,  2.75it/s]


 60%|█████▉    | 2019/3370 [07:43<06:52,  3.28it/s]


 60%|█████▉    | 2021/3370 [07:45<12:00,  1.87it/s]


 60%|██████    | 2025/3370 [07:47<10:39,  2.10it/s]


 60%|██████    | 2029/3370 [07:48<09:36,  2.33it/s]


 60%|██████    | 2033/3370 [07:49<08:49,  2.52it/s]


 60%|██████    | 2037/3370 [07:51<08:23,  2.65it/s]


 61%|██████    | 2041/3370 [07:53<09:30,  2.33it/s]


 61%|██████    | 2045/3370 [07:54<09:18,  2.37it/s]


 61%|██████    | 2049/3370 [07:56<09:19,  2.36it/s]


 61%|██████    | 2053/3370 [07:58<09:00,  2.44it/s]


 61%|██████    | 2057/3370 [07:59<08:32,  2.56it/s]


 61%|██████    | 2061/3370 [

 79%|███████▉  | 2656/3370 [11:41<06:47,  1.75it/s]


 79%|███████▉  | 2659/3370 [11:41<05:31,  2.15it/s]


 79%|███████▉  | 2660/3370 [11:42<06:12,  1.91it/s]


 79%|███████▉  | 2663/3370 [11:43<05:10,  2.28it/s]


 79%|███████▉  | 2664/3370 [11:44<06:31,  1.80it/s]


 79%|███████▉  | 2667/3370 [11:44<05:21,  2.18it/s]


 79%|███████▉  | 2668/3370 [11:45<06:40,  1.75it/s]


 79%|███████▉  | 2671/3370 [11:46<05:34,  2.09it/s]


 79%|███████▉  | 2672/3370 [11:47<07:11,  1.62it/s]


 79%|███████▉  | 2675/3370 [11:47<05:15,  2.20it/s]


 79%|███████▉  | 2676/3370 [11:48<07:03,  1.64it/s]


 79%|███████▉  | 2678/3370 [11:48<05:11,  2.22it/s]


 79%|███████▉  | 2679/3370 [11:49<07:34,  1.52it/s]


 80%|███████▉  | 2680/3370 [11:50<06:46,  1.70it/s]


 80%|███████▉  | 2682/3370 [11:50<05:16,  2.17it/s]


 80%|███████▉  | 2683/3370 [11:51<06:15,  1.83it/s]


 80%|███████▉  | 2684/3370 [11:51<06:12,  1.84it/s]


 80%|███████▉  | 2686/3370 [11:52<04:34,  2.49it/s]


 80%|███████▉  | 2687/3370 [

 87%|████████▋ | 2936/3370 [13:25<02:26,  2.97it/s]


 87%|████████▋ | 2937/3370 [13:25<02:16,  3.17it/s]


 87%|████████▋ | 2938/3370 [13:25<02:14,  3.21it/s]


 87%|████████▋ | 2940/3370 [13:26<02:29,  2.88it/s]


 87%|████████▋ | 2941/3370 [13:26<02:11,  3.26it/s]


 87%|████████▋ | 2942/3370 [13:28<04:00,  1.78it/s]


 87%|████████▋ | 2945/3370 [13:28<03:10,  2.23it/s]


 87%|████████▋ | 2946/3370 [13:29<03:48,  1.85it/s]


 87%|████████▋ | 2948/3370 [13:30<03:29,  2.01it/s]


 88%|████████▊ | 2950/3370 [13:30<03:11,  2.19it/s]


 88%|████████▊ | 2952/3370 [13:31<02:48,  2.48it/s]


 88%|████████▊ | 2953/3370 [13:31<02:44,  2.53it/s]


 88%|████████▊ | 2954/3370 [13:32<02:55,  2.37it/s]


 88%|████████▊ | 2956/3370 [13:32<02:41,  2.57it/s]


 88%|████████▊ | 2957/3370 [13:33<02:30,  2.74it/s]


 88%|████████▊ | 2958/3370 [13:33<02:43,  2.51it/s]


 88%|████████▊ | 2960/3370 [13:34<02:28,  2.76it/s]


 88%|████████▊ | 2961/3370 [13:34<02:10,  3.13it/s]


 88%|████████▊ | 2962/3370 [

 98%|█████████▊| 3311/3370 [15:44<00:30,  1.94it/s]


 98%|█████████▊| 3312/3370 [15:44<00:22,  2.54it/s]


 98%|█████████▊| 3313/3370 [15:44<00:19,  2.93it/s]


 98%|█████████▊| 3314/3370 [15:45<00:19,  2.85it/s]


 98%|█████████▊| 3315/3370 [15:46<00:30,  1.82it/s]


 98%|█████████▊| 3316/3370 [15:46<00:28,  1.87it/s]


 98%|█████████▊| 3317/3370 [15:46<00:22,  2.39it/s]


 98%|█████████▊| 3319/3370 [15:48<00:24,  2.10it/s]


 99%|█████████▊| 3320/3370 [15:48<00:20,  2.41it/s]


 99%|█████████▊| 3321/3370 [15:48<00:16,  3.06it/s]


 99%|█████████▊| 3323/3370 [15:49<00:17,  2.71it/s]


 99%|█████████▊| 3325/3370 [15:49<00:15,  2.88it/s]


 99%|█████████▊| 3327/3370 [15:50<00:16,  2.54it/s]


 99%|█████████▉| 3328/3370 [15:51<00:16,  2.61it/s]


 99%|█████████▉| 3329/3370 [15:51<00:12,  3.20it/s]


 99%|█████████▉| 3331/3370 [15:53<00:18,  2.07it/s]


 99%|█████████▉| 3335/3370 [15:54<00:15,  2.28it/s]


 99%|█████████▉| 3337/3370 [15:55<00:13,  2.45it/s]


 99%|█████████▉| 3339/3370 [

In [43]:
v_dataset = experiment.validation_dataset
v_dataloader = experiment.get_dataloader(v_dataset, shuffle=False)
v_dataset.return_indices = True

In [49]:
v_filenames = []
for j, batch in enumerate(v_dataloader[0]):
    img, gt, idx = batch
    for i in idx:
        v_filenames.append(v_dataset.filename(i))

In [51]:
any([_ in filenames for _ in v_filenames])

False

In [47]:
v_filenames

['CNV-7873827-41.jpeg',
 'DME-1896542-2.jpeg',
 'NORMAL-1558951-5.jpeg',
 'NORMAL-2095710-14.jpeg',
 'CNV-7315522-181.jpeg',
 'NORMAL-3059523-6.jpeg',
 'DRUSEN-9540117-11.jpeg',
 'NORMAL-4533877-1.jpeg',
 'NORMAL-7663507-1.jpeg',
 'CNV-2025105-25.jpeg',
 'CNV-9374492-231.jpeg',
 'NORMAL-3996434-2.jpeg',
 'NORMAL-6418744-21.jpeg',
 'NORMAL-1428906-4.jpeg',
 'NORMAL-1067440-22.jpeg',
 'CNV-4308224-1.jpeg',
 'NORMAL-3676009-48.jpeg',
 'CNV-9734808-36.jpeg',
 'CNV-6668596-177.jpeg',
 'NORMAL-9054040-25.jpeg',
 'CNV-8728968-150.jpeg',
 'DRUSEN-8986660-76.jpeg',
 'DME-1608084-4.jpeg',
 'NORMAL-3929912-24.jpeg',
 'CNV-4708782-30.jpeg',
 'NORMAL-8907654-5.jpeg',
 'CNV-1112835-247.jpeg',
 'NORMAL-9699329-32.jpeg',
 'NORMAL-3116592-6.jpeg',
 'NORMAL-2628737-9.jpeg',
 'NORMAL-6434323-23.jpeg',
 'NORMAL-459672-2.jpeg',
 'CNV-624911-4.jpeg',
 'NORMAL-5571744-33.jpeg',
 'NORMAL-3488701-4.jpeg',
 'NORMAL-4357252-16.jpeg',
 'NORMAL-3574650-4.jpeg',
 'CNV-7565927-179.jpeg',
 'NORMAL-2662105-7.jpeg',
 '